In [1]:
import torch
import numpy as np
import pandas as pd
import codecs
import re
from collections import Counter
import nltk
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer

### 0.Preprocessing

In [2]:
# Encoding in windows-1252, utf-8 generate error on some char
file = codecs.open("train_all.label", "r+","windows-1252")
data = []
list_of_words = []
for line in file.readlines():
    for word in line.split():
        list_of_words.append(word)
    data.append(line)

In [3]:
# Divided file into 2 list:
# questions = list of questions 
# labels = list of labels

questions = []
labels = []

(data[0]).split()[0]
for string in data:
    labels.append(string.split()[0])
    questions.append(string.split()[1:])
len(questions)

15452

In [4]:
# Lemmatisation
# Je travaille sur list_of_words
def lemm(phrase):
    lemmer = WordNetLemmatizer()
    lemmed = []
    for word in phrase:
        lemmed.append(lemmer.lemmatize(word, pos="v"))
    return lemmed

In [5]:
list_of_lemms = lemm(list_of_words)

In [6]:
# Method to add tags begin and end to phrases list.
# WARNING : this method need to be executed only ONE time.
def add_tag():
    for i in range(0, len(questions)):
        questions[i].insert(0, '<bos>')
        questions[i].append('<eos>')

In [7]:
# Using sklearn to get the vocabulary
vectorizer = CountVectorizer()
str_questions = []
for quest in questions:
    str_questions.append(' '.join(quest))
output = vectorizer.fit(str_questions)

In [8]:
# This is the vocabulary dict.
vocabulary = output.vocabulary_

In [9]:
# print(output.vocabulary_)
# This allow to get the all vectors of vocabulary
# 15452 vectors of size 8413
# corresponding to 15452 questions and 8413 words
vectors = vectorizer.transform(str_questions)
vector_list = vectors.toarray()
vector_list

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [10]:
# Method to get the binary word_vector from the vocabulary.
def get_word_vector(word):
    return vectorizer.transform([word]).toarray()